# 介绍

`AutoGen` 是一个LLM应用框架，主要用于多 agent 协作解决复杂任务

In [1]:
## 安装 autogen
%pip install pyautogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 483.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 663.2 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 764.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 795.8 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
## Local execution

import autogen
from autogen import AssistantAgent, UserProxyAgent
import os

llm_config = {"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}
assistant = AssistantAgent("assistant", llm_config=llm_config)

user_proxy = UserProxyAgent(
    "user_proxy", code_execution_config={"executor": autogen.coding.LocalCommandLineCodeExecutor(work_dir="coding")}
)

# Start the chat
user_proxy.initiate_chat(
    assistant,
    message="Plot a chart of NVDA and TESLA stock price change YTD.",
)


user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

Certainly. Here's a python script that will pull the Year-to-Date (YTD) stock data for NVDA (NVIDIA Corporation) and TESLA using the `yfinance` library, which downloads stock data from Yahoo Finance. Then, it will plot the data using the `matplotlib` library.

```python
# filename: stock_chart.py

import yfinance as yf
import matplotlib.pyplot as plt

# Fetch the YTD data for NVDA and TESLA
nvda_data = yf.Ticker("NVDA").history('YTD')
tesla_data = yf.Ticker("TSLA").history('YTD')

# Plot NVDA data
plt.plot(nvda_data.index, nvda_data['Close'], label='NVDA')

# Plot TESLA data
plt.plot(tesla_data.index, tesla_data['Close'], label='TESLA')

plt.xlabel('Date')
plt.ylabel('Closing Price (USD)')
plt.title('Stock price change YTD of NVDA and TESLA')
plt.legend()
plt.show()
```

Before running this scrip

ChatResult(chat_id=None, chat_history=[{'content': 'Plot a chart of NVDA and TESLA stock price change YTD.', 'role': 'assistant'}, {'content': 'Certainly. Here\'s a python script that will pull the Year-to-Date (YTD) stock data for NVDA (NVIDIA Corporation) and TESLA using the `yfinance` library, which downloads stock data from Yahoo Finance. Then, it will plot the data using the `matplotlib` library.\n\n```python\n# filename: stock_chart.py\n\nimport yfinance as yf\nimport matplotlib.pyplot as plt\n\n# Fetch the YTD data for NVDA and TESLA\nnvda_data = yf.Ticker("NVDA").history(\'YTD\')\ntesla_data = yf.Ticker("TSLA").history(\'YTD\')\n\n# Plot NVDA data\nplt.plot(nvda_data.index, nvda_data[\'Close\'], label=\'NVDA\')\n\n# Plot TESLA data\nplt.plot(tesla_data.index, tesla_data[\'Close\'], label=\'TESLA\')\n\nplt.xlabel(\'Date\')\nplt.ylabel(\'Closing Price (USD)\')\nplt.title(\'Stock price change YTD of NVDA and TESLA\')\nplt.legend()\nplt.show()\n```\n\nBefore running this script, ma

In [6]:
## Docker exceution

import autogen
from autogen import AssistantAgent, UserProxyAgent

llm_config = {"model": "gpt-4", "api_key": os.environ["OPENAI_API_KEY"]}

with autogen.coding.DockerCommandLineCodeExecutor(work_dir="coding") as code_executor:
    assistant = AssistantAgent("assistant", llm_config=llm_config)
    user_proxy = UserProxyAgent(
        "user_proxy", code_execution_config={"executor": code_executor}
    )

    # Start the chat
    user_proxy.initiate_chat(
        assistant,
        message="Plot a chart of NVDA and TESLA stock price change YTD. Save the plot to a file called plot.png",
    )
    

DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [9]:
# Install
## pip install pyautogen
## Install Docker for Code Execution

from pathlib import Path
from autogen import UserProxyAgent
from autogen.coding import DockerCommandLineCodeExecutor

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)

with DockerCommandLineCodeExecutor(work_dir=work_dir) as code_executor:
    user_proxy = UserProxyAgent(
        name="user_proxy",
        code_execution_config={"executor": code_executor},
    )

In [10]:
## 配置`ConversableAgent`

import os

from autogen import ConversableAgent

agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

In [11]:
## 使用`ConversableAgent`生成回复
reply = agent.generate_reply(messages=[{"content": "Tell me a joke.", "role": "user"}])
print(reply)


Sure, here's a classic for you:

Why don't scientists trust atoms?

Because they make up everything!


In [12]:
## 使用多个`ConversableAgent`创建角色和对话

cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.9, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": [{"model": "gpt-4", "temperature": 0.7, "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [13]:
## 参与对话
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=2)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's one for you:

Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, good one Cathy! Now, let me share one with you.

Why don't we ever tell secrets on a farm?

Because the potatoes have eyes, the corn has ears, and the beans stalk.

--------------------------------------------------------------------------------
cathy (to joe):

Hahaha, that's a great one! Love a good farm joke. Now, I see why we make such a great comedy duo!

--------------------------------------------------------------------------------
